1. Defining Models

In [2]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str

2. Validation

In [6]:
from pydantic import ValidationError

try:
    user = User(id=1, name="Alice", email="not an email")
except ValidationError as e:
    print(e)

3. Serialization

In [5]:
user = User(id=1, name="Alice", email="alice@example.com")
# print(user.json())
print(user.model_dump_json())

{"id":1,"name":"Alice","email":"alice@example.com"}


4. Deserialization


In [8]:

user_dict = {"id": 1, "name": "Alice", "email": "alice@example.com"}
user = User(**user_dict)

5. Configuration

In [10]:
class User(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        json_schema_extra = {
            "example": {
                "id": 1,
                "name": "Alice",
                "email": "alice@example.com"
            }
        }

6. Relationships with SQLAlchemy

In [23]:
from sqlalchemy.orm import Session
from pydantic import BaseModel
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table

class UserModel(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        # orm_mode = True
        from_attributes = True
engine = create_engine("sqlite:///example.db")
# Base.metadata.create_all(engine)
# Assuming you have a SQLAlchemy session and a User instance
session = Session(engine)
user = session.query(User).filter(User.email == "alice@example.com").first()

# Convert the SQLAlchemy model to a Pydantic model for serialization
# user_model = UserModel.from_orm(user)
user_model = UserModel.from_orm(user)

ArgumentError: Column expression, FROM clause, or other columns clause element expected, got <class '__main__.User'>.

In [29]:
from sqlalchemy import create_engine, Column, Integer, String
# from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session, declarative_base
from pydantic import BaseModel

# Define the SQLAlchemy model
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True, index=True)
    name = Column(String, index=True)
    email = Column(String, unique=True, index=True)

# Define the Pydantic model
class UserModel(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        from_attributes = True

# Create the database
engine = create_engine("sqlite:///example.db")
Base.metadata.create_all(engine)

# Assuming you have a SQLAlchemy session and a User instance
session = Session(engine)
user = session.query(User).filter(User.email == "alice@example.com").first()

# Convert the SQLAlchemy model to a Pydantic model for serialization
user_model = UserModel.model_validate(user)
print(user_model)

user = session.query(User).filter(User.name == "Alice").first()

# Convert the SQLAlchemy model to a Pydantic model for serialization
user_model = UserModel.model_validate(user)
print(user_model,user)

id=1 name='Alice' email='alice@example.com'
id=1 name='Alice' email='alice@example.com' <__main__.User object at 0x000001AF7D5D9890>


7. Advanced Validation

In [16]:


from pydantic import BaseModel, validator, field_validator

class User(BaseModel):
    name: str
    age: int

    # @validator('age')
    @field_validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('Age must be 18 or older')
        return v

8. Custom Data Types


In [18]:
from pydantic import BaseModel, Field
from typing import List

class User(BaseModel):
    name: str
    friends: List[str] = Field(default_factory=list)



9. Environment Variables


In [20]:
# from pydantic import BaseSettings
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    database_url: str

settings = Settings()

ValidationError: 1 validation error for Settings
database_url
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.4/v/missing

10. Recursive Models


In [17]:

from pydantic import BaseModel
from typing import List

class TreeNode(BaseModel):
    value: int
    children: List['TreeNode'] = []

In [1]:

from IPython.display import display, Markdown

text = """Given the dynamic nature of software development and the rapid pace of updates, it's challenging to cover all possibilities with Pydantic in a single response. However, I can provide you with a comprehensive overview of key features and best practices for using Pydantic, including handling models, validation, and serialization. This should give you a solid foundation to explore further on your own.

### 1. Defining Models

Pydantic models are defined using Python classes that inherit from `pydantic.BaseModel`. Each attribute of the class represents a field in the model, and you can specify types, default values, and more.

```python
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str
```

### 2. Validation

Pydantic automatically validates input data against the model's schema. If the data is invalid, Pydantic raises a `ValidationError`.

```python
from pydantic import ValidationError

try:
    user = User(id=1, name="Alice", email="not an email")
except ValidationError as e:
    print(e)
```

### 3. Serialization

Pydantic models can be easily serialized to JSON, dictionaries, and more.

```python
user = User(id=1, name="Alice", email="alice@example.com")
print(user.json())
```

### 4. Deserialization

You can also create Pydantic models from dictionaries or JSON strings.

```python
user_dict = {"id": 1, "name": "Alice", "email": "alice@example.com"}
user = User(**user_dict)
```

### 5. Configuration

Pydantic models can be customized using the `Config` class. This includes setting default values, customizing field names, and more.

```python
class User(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "name": "Alice",
                "email": "alice@example.com"
            }
        }
```

### 6. Relationships with SQLAlchemy

Pydantic can work seamlessly with SQLAlchemy models to serialize and deserialize data.

```python
from sqlalchemy.orm import Session
from pydantic import BaseModel

class UserModel(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        orm_mode = True

# Assuming you have a SQLAlchemy session and a User instance
session = Session(engine)
user = session.query(User).filter(User.email == "alice@example.com").first()

# Convert the SQLAlchemy model to a Pydantic model for serialization
user_model = UserModel.from_orm(user)
```

### 7. Advanced Validation

Pydantic allows for complex validation rules using the `@validator` decorator.

```python
from pydantic import BaseModel, validator

class User(BaseModel):
    name: str
    age: int

    @validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('Age must be 18 or older')
        return v
```

### 8. Custom Data Types

You can define custom data types and use them in your Pydantic models.

```python
from pydantic import BaseModel, Field
from typing import List

class User(BaseModel):
    name: str
    friends: List[str] = Field(default_factory=list)
```

### 9. Environment Variables

Pydantic can automatically populate models from environment variables.

```python
from pydantic import BaseSettings

class Settings(BaseSettings):
    database_url: str

settings = Settings()
```

### 10. Recursive Models

Pydantic supports recursive models, which are useful for nested data structures.

```python
from pydantic import BaseModel
from typing import List

class TreeNode(BaseModel):
    value: int
    children: List['TreeNode'] = []
```

This overview covers many of the key features and best practices for using Pydantic. For more advanced use cases, including custom validators, dynamic models, and more, refer to the [Pydantic documentation](https://pydantic-docs.helpmanual.io/)."""
display(Markdown(text))

Given the dynamic nature of software development and the rapid pace of updates, it's challenging to cover all possibilities with Pydantic in a single response. However, I can provide you with a comprehensive overview of key features and best practices for using Pydantic, including handling models, validation, and serialization. This should give you a solid foundation to explore further on your own.

### 1. Defining Models

Pydantic models are defined using Python classes that inherit from `pydantic.BaseModel`. Each attribute of the class represents a field in the model, and you can specify types, default values, and more.

```python
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str
```

### 2. Validation

Pydantic automatically validates input data against the model's schema. If the data is invalid, Pydantic raises a `ValidationError`.

```python
from pydantic import ValidationError

try:
    user = User(id=1, name="Alice", email="not an email")
except ValidationError as e:
    print(e)
```

### 3. Serialization

Pydantic models can be easily serialized to JSON, dictionaries, and more.

```python
user = User(id=1, name="Alice", email="alice@example.com")
print(user.json())
```

### 4. Deserialization

You can also create Pydantic models from dictionaries or JSON strings.

```python
user_dict = {"id": 1, "name": "Alice", "email": "alice@example.com"}
user = User(**user_dict)
```

### 5. Configuration

Pydantic models can be customized using the `Config` class. This includes setting default values, customizing field names, and more.

```python
class User(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        schema_extra = {
            "example": {
                "id": 1,
                "name": "Alice",
                "email": "alice@example.com"
            }
        }
```

### 6. Relationships with SQLAlchemy

Pydantic can work seamlessly with SQLAlchemy models to serialize and deserialize data.

```python
from sqlalchemy.orm import Session
from pydantic import BaseModel

class UserModel(BaseModel):
    id: int
    name: str
    email: str

    class Config:
        orm_mode = True

# Assuming you have a SQLAlchemy session and a User instance
session = Session(engine)
user = session.query(User).filter(User.email == "alice@example.com").first()

# Convert the SQLAlchemy model to a Pydantic model for serialization
user_model = UserModel.from_orm(user)
```

### 7. Advanced Validation

Pydantic allows for complex validation rules using the `@validator` decorator.

```python
from pydantic import BaseModel, validator

class User(BaseModel):
    name: str
    age: int

    @validator('age')
    def check_age(cls, v):
        if v < 18:
            raise ValueError('Age must be 18 or older')
        return v
```

### 8. Custom Data Types

You can define custom data types and use them in your Pydantic models.

```python
from pydantic import BaseModel, Field
from typing import List

class User(BaseModel):
    name: str
    friends: List[str] = Field(default_factory=list)
```

### 9. Environment Variables

Pydantic can automatically populate models from environment variables.

```python
from pydantic import BaseSettings

class Settings(BaseSettings):
    database_url: str

settings = Settings()
```

### 10. Recursive Models

Pydantic supports recursive models, which are useful for nested data structures.

```python
from pydantic import BaseModel
from typing import List

class TreeNode(BaseModel):
    value: int
    children: List['TreeNode'] = []
```

This overview covers many of the key features and best practices for using Pydantic. For more advanced use cases, including custom validators, dynamic models, and more, refer to the [Pydantic documentation](https://pydantic-docs.helpmanual.io/).